## Model Grafting
Explored the possibility of optimizing over the attention and mlp modules in a gpt2 model to find the minimal set of modules which can recover performance of the full model. 

Inspired from https://arxiv.org/pdf/2302.06600

Did not work. 

In [1]:
try:
    import google.colab
    IN_COLAB = True
    # %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    renderer = "colab"
except:
    IN_COLAB = False
    from IPython import get_ipython
    %load_ext autoreload
    %autoreload 2
    renderer = "jupyterlab"

In [2]:
%%bash
cd ../
pip install poetry
poetry install
cd notebooks

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached platformdirs-3.10.0-py3-none-any.whl (17 kB)
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.8.0
    Uninstalling platformdirs-3.8.0:
      Successfully uninstalled platformdirs-3.8.0
Installing dependencies from lock file

Package operations: 0 installs, 1 update, 0 removals

  • Downgrading platformdirs (3.10.0 -> 3.8.0)

Installing the current project: transformer-lens (0.0.0)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.167.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 5.2.0 which is incompatible.
sagemaker 2.167.0 requires protobuf<4.0,>=3.1, but you have protobuf 4.23.3 which is incompatible.


In [4]:
!pip install 'numpy<1.24'

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [5]:
import types
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader
import json
from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy
import subprocess

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
from tqdm import tqdm

import ast
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import torch
from matplotlib.cm import ScalarMappable
import pickle

if IN_COLAB: 
    import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens.utilities import devices
from transformer_lens.past_key_value_caching import HookedTransformerKeyValueCache
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache



Matplotlib is building the font cache; this may take a moment.


In [6]:
import torch.nn.functional as F
import torch.optim as optim


In [7]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def cuda():
    return torch.cuda.is_available()

def get_device(): 
    return "cuda" if cuda() else "cpu"

def save_pickle(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)

# Function to load a pickle object from a file
def load_pickle(filename):
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj

def save_json(data, file_path):
    with open(file_path, "w") as json_file:
        json.dump(data, json_file, indent=4)

def load_json(file_path):
    with open(file_path, "r") as json_file:
        loaded_data = json.load(json_file)
    return loaded_data

device = get_device()
device

'cuda'

## Model

- fold_ln: Whether to fold in the LayerNorm weights to the subsequent linear layer. This does not change the computation.
- center_writing_weights: Whether to center weights writing to the residual stream (ie set mean to be zero). Due to LayerNormthis doesn't change the computation.
- center_unembed : Whether to center W_U (ie set mean to be zero). Softmax is translation invariant so this doesn't affect log probs or loss, but does change logits. Defaults to True.
- refactor_factored_attn_matrices: Whether to convert the factoredmatrices (W_Q & W_K, and W_O & W_V) to be "even". 

In [8]:
MODEL_NAME = "gpt2-large"
model = HookedTransformer.from_pretrained(
        MODEL_NAME,
        center_unembed=True,  
        center_writing_weights=True,              # Whether to center weights writing to the residual stream (ie set mean to be zero). Due to LayerNorm this doesn't change the computation.      
        fold_ln=True,                             # Whether to  fold in the LayerNorm weights to the subsequent linear layer.
        refactor_factored_attn_matrices=True,
    )


 

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


## Dataset
We use the CounterFact Dataset. We subsample facts which the model can accurately predict. 

In [9]:
dataset = load_json("data/fact_dataset.json")


def sample_dataset(dataset, idx = None): 
    if idx is None: 
        prompt = "The {} is located in"
        subject = "Eiffel Tower"
        target = "Paris"
    else: 
        sample = dataset[idx]
        prompt = sample["requested_rewrite"]["prompt"]
        subject = sample["requested_rewrite"]["subject"]
        target = sample["requested_rewrite"]["target_true"]["str"]
    return prompt, subject, target


In [10]:
sample = sample_dataset(dataset)
prompt, subject, target = sample
prompt = prompt.format(subject)

## Ablation Methods: 
- Noise ablation 
- Resample ablation

In [11]:
def noise_ablation(prompt, subject, target, n_noise_samples=5, vx=3):
    subject_tokens = model.to_tokens(subject)
    
    #shape: batch, n_tokens, embedding_dim
    subject_embedding = model.embed(subject_tokens)
    _, n_tokens, embedding_dim = subject_embedding.shape
    
    #noise: N(0,v), v = 3*std(embedding)
    embedding = model.W_E
    v = vx*torch.std(embedding, dim=0) #for each v in V
    noise = torch.randn(
        (n_noise_samples, n_tokens, embedding_dim)
    ).to(device) + v
    
    subject_embedding_w_noise = subject_embedding + noise
    
    #shape: batch, n_tokens, vocab_size (logits)
    unembedded_subject = model.unembed(subject_embedding_w_noise)

    noisy_subject_tokens = torch.argmax(unembedded_subject, dim=-1)
    noisy_subject_str = [
        model.to_string(nst) for nst in noisy_subject_tokens
    ]
    true_prompt = prompt.format(subject)
    corrupted_prompts = [
        prompt.format(nss) for nss in noisy_subject_str
    ]
    return true_prompt, corrupted_prompts, target

def resample_ablation(prompt, subject, target, n_noise_samples=20):
    subject_tokens = model.to_tokens(subject)
    embedding = model.W_E
    #we select n random rows from the embedding matrix
    permutations = torch.randperm(embedding.size(0))[:n_noise_samples]
    random_samples = embedding[permutations]
    #unsqueeze a token dimension between batch and embedding dims
    random_samples = random_samples.unsqueeze(dim=1)
    #we de-embed these rows
    random_embeddings = model.unembed(random_samples)
    random_tokens = torch.argmax(random_embeddings, dim=-1)
    random_subject_str = [
        model.to_string(t) for t in random_tokens
    ]
    corrupted_facts = [
        prompt.format(s) for s in random_subject_str
    ]
    true_fact = prompt.format(subject)
    return true_fact, corrupted_facts, target
    

    
    

In [12]:
prompt, subject, target = sample_dataset(dataset)
noise_ablation(prompt, subject, target)

('The Eiffel Tower is located in',
 ['The  mechanersonocularese Sites is located in',
  'The ungle Presenceteinfu STA is located in',
  'The apesengedruciatinghidRECT is located in',
  'The untaatistre HELillary is located in',
  'The icidesetsk Lie aisle Pirates is located in'],
 'Paris')

In [10]:
def pad_from_left(tokens : torch.tensor, maxlen:int):
    pad_token = model.tokenizer.pad_token_id
    padded_tokenized_inputs = torch.zeros(tokens.shape[0], maxlen)
    
    n_pads = maxlen - tokens.shape[-1]
    padded_tokenized_inputs[:,n_pads] = pad_token
    padded_tokenized_inputs[:,n_pads:] = tokens
    return padded_tokenized_inputs.long()

def pad_to_same_length(clean_tokens, corrupted_tokens): 
    
    maxlen = max([clean_tokens.shape[-1], corrupted_tokens.shape[-1]])
    
    if clean_tokens.shape[-1] > corrupted_tokens.shape[-1]: 
        corrupted_tokens = pad_from_left(corrupted_tokens, maxlen)
    elif clean_tokens.shape[-1] < corrupted_tokens.shape[-1]: 
        clean_tokens = pad_from_left(clean_tokens, maxlen)
    return clean_tokens, corrupted_tokens

In [11]:
possible_patches = [
    # 'ln1.hook_scale',
    # 'ln1.hook_normalized',
    # 'attn.hook_q',
    # 'attn.hook_k',
    # 'attn.hook_v',
    # 'attn.hook_attn_scores',
    # 'attn.hook_pattern',
    # 'attn.hook_z',
    'hook_attn_out',
    # 'hook_resid_mid',
    # 'ln2.hook_scale',
    # 'ln2.hook_normalized',
    # 'mlp.hook_pre',
    # 'mlp.hook_post',
    'hook_mlp_out',
    # 'hook_resid_post'
]

In [12]:
def patch(
    corrupted_residual_component: Float[torch.Tensor, "batch pos d_model"],
    hook,
    pos,
    cache):
    corrupted_residual_component[:, pos, :] = cache[hook.name][:, pos, :]
    """
    Restore a patch in clean run. 
    """
    return corrupted_residual_component

def decompose_residual(clean_cache,
                  corrupted_cache,
                  token_idx,
                  target_token_idx): 
    #shape: [74, 4, 9, 50257] = n_layers, batch, tokens, vocab_size
    residual_clean_stack, labels = clean_cache.decompose_resid(layer=-1, return_labels=True)       
    residual_corrupted_stack = corrupted_cache.decompose_resid(layer=-1, return_labels=False)
   
    token_idx_expanded = token_idx.repeat(residual_clean_stack.shape[0],1,1)
    
    residual_clean_stack = unembedding_function(residual_clean_stack, cache)
    residual_clean_stack = residual_clean_stack[:,:,-1,:]
    residual_clean_logits = residual_clean_stack.gather(index=target_idx_expanded, dim=-1) - residual_clean_stack.mean(dim=-1, keepdim=True)
    
    residual_corrupted_stack = unembedding_function(residual_corrupted_stack, cache)
    residual_corrupted_stack = residual_corrupted_stack[:,:,-1,:]
    #shape: layers, batch
    residual_corrupted_logits = residual_corrupted_stack.gather(index=target_idx_expanded, dim=-1) - residual_clean_stack.mean(dim=-1, keepdim=True)
    
    direct_effect = (residual_clean_logits - residual_corrupted_logits).mean(dim=-1)
    print(direct_effect.shape)
    raise
    return direct_effect

    


def get_causal_positions(original_fact, corrupted_facts, target, use_mle_idx=True): 
    
    clean_tokens = model.to_tokens(clean_prompt, prepend_bos=True) 
    corrupted_tokens = model.to_tokens(corrupted_facts, prepend_bos=True)
    #for now, use single corruption
    clean_tokens, corrupted_tokens = pad_to_same_length(clean_tokens, corrupted_tokens)
    corrupted_tokens = corrupted_tokens[0].unsqueeze(0)
    
    target_token_idx = target_token_idx.expand(corrupted_tokens.shape[0], -1)
    n_tokens = clean_tokens.shape[-1]

    clean_logits, clean_cache = model.run_with_cache(clean_tokens, return_type="logits")
    _, corrupted_cache = model.run_with_cache(corrupted_tokens, return_type="logits")
    
    clean_logits = clean_logits[:,-1,:]
    
    mle_token_idx = clean_logits.argmax(dim=-1)
    token_idx = mke_token_idx if use_mle_idx else target_token_idx
    
   
  
    layer_names = []
    for i in range(n_layers): 
        patch_name = f"blocks.{i}.resid_post"
        
        hook_fn = partial(patch, pos=tok_pos, cache=clean_cache)
        hook = (patch_name, hook_fn)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[hook],
            return_type="logits"
        )   
        patched_logits = patched_logits[:,-1,:]
        patched_logits = patched_logits - patched_logits.mean(dim=-1, keepdim=True)
        patched_loss = F.softmax(patched_logits, dim=-1)

        causal_positions_logit_diff[i, tok_pos] = (patched_logits[:,target_idx] - corrupted_logits[:,target_idx]).to("cpu")
        causal_positions_prob_diff[i, tok_pos] = (patched_logits[:,target_idx] - corrupted_prob[:,target_idx]).to("cpu")
        causal_positions_KL_loss[i, tok_pos] = F.kl_div(patched_logits.log(), corrupted_prob, reduction='batchmean').to("cpu")
        causal_positions_CE_loss_diff[i, tok_pos] = F.cross_entropy(patched_logits, corrupted_logits).to("cpu")

    return {
        "causal_positions_logit_diff": causal_positions_logit_diff.to("cpu"), 
        "causal_positions_prob_diff": causal_positions_prob_diff.to("cpu"),
        "causal_positions_KL_loss": causal_positions_KL_loss.to("cpu"),
        "causal_positions_CE_loss_diff": causal_positions_CE_loss_diff.to("cpu"),
    }

In [13]:
clean_prompt, subject, target = sample_dataset(dataset)
original_fact, corrupted_facts, target = resample_ablation(clean_prompt, subject, target, n_noise_samples=10)

graft = get_causal_positions(original_fact, corrupted_facts, target)

TypeError: get_causal_positions() missing 1 required positional argument: 'target'

In [ ]:
clean_logits, clean_cache = model.run_with_cache(clean_prompt

In [14]:
def patch(
    corrupted_residual_component: Float[torch.Tensor, "batch pos d_model"],
    hook,
    pos,
    cache):
    corrupted_residual_component[:, pos, :] = cache[hook.name][:, pos, :]
    """
    Restore a patch in clean run. 
    """
    return corrupted_residual_component


def get_causal_positions(model, original_fact, corrupted_facts, target): 
    
    clean_tokens = model.to_tokens(clean_prompt, prepend_bos=True) 
    corrupted_tokens = model.to_tokens(corrupted_facts, prepend_bos=True)
    #for now, use single corruption
    clean_tokens, corrupted_tokens = pad_to_same_length(clean_tokens, corrupted_tokens)
    corrupted_tokens = corrupted_tokens[0].unsqueeze(0)
    n_tokens = clean_tokens.shape[-1]

    clean_logits, clean_cache = model.run_with_cache(clean_tokens, return_type="logits")
    corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens, return_type="logits")
    clean_logits = clean_logits[:,-1,:] ; corrupted_logits = corrupted_logits[:,-1,:]
    clean_logits = clean_logits - clean_logits.mean(dim=-1, keepdim=True)
    corrupted_logits = corrupted_logits - corrupted_logits.mean(dim=-1, keepdim=True)
    
    mle_idx = torch.argmax(clean_logits, dim=-1)
    clean_prob = F.softmax(clean_logits, dim=-1)
    corrupted_prob = F.softmax(corrupted_logits, dim=-1)
    
    #calculate_total_effects
    overall_total_effect_logits = clean_logits[:,mle_idx] - corrupted_logits[:,mle_idx]
    overall_total_effect_prob = clean_prob[:,mle_idx] - corrupted_prob[:,mle_idx]
    overall_total_effect_kl = F.kl_div(clean_prob.log(), corrupted_prob, reduction='batchmean')
    overall_total_effect_CE = F.cross_entropy(clean_logits, corrupted_logits)

    n_layers = model.cfg.n_layers
    
    causal_positions_CE_loss_diff = torch.zeros((n_layers*2, n_tokens))
    causal_positions_KL_loss = torch.zeros((n_layers*2, n_tokens))
    causal_positions_logit_diff = torch.zeros((n_layers*2, n_tokens))
    causal_positions_prob_diff = torch.zeros((n_layers*2, n_tokens))
    
  
    layer_names = []
    for i in range(n_layers*2): 
        if i%2==0: 
            layer_names.append(f"attn_{i//2}")
            patch_name = f"blocks.{i//2}.hook_attn_out"
        else: 
            layer_names.append(f"mlp_{i//2}")
            patch_name = f"blocks.{i//2}.hook_mlp_out"
                    

        for tok_pos in range(n_tokens): 
            hook_fn = partial(patch, pos=tok_pos, cache=clean_cache)
            hook = (patch_name, hook_fn)
            patched_logits = model.run_with_hooks(
                corrupted_tokens,
                fwd_hooks=[hook],
                return_type="logits"
            )   
            patched_logits = patched_logits[:,-1,:]
            patched_logits = patched_logits - patched_logits.mean(dim=-1, keepdim=True)
            patched_loss = F.softmax(patched_logits, dim=-1)
            
         
            causal_positions_logit_diff[i, tok_pos] = (patched_logits[:,mle_idx] - corrupted_logits[:,mle_idx]).to("cpu")
            causal_positions_prob_diff[i, tok_pos] = (patched_logits[:,mle_idx] - corrupted_prob[:,mle_idx]).to("cpu")
            causal_positions_KL_loss[i, tok_pos] = F.kl_div(patched_logits.log(), corrupted_prob, reduction='batchmean').to("cpu")
            causal_positions_CE_loss_diff[i, tok_pos] = F.cross_entropy(patched_logits, corrupted_logits).to("cpu")


    return {
        "causal_positions_logit_diff": causal_positions_logit_diff.to("cpu"), 
        "causal_positions_prob_diff": causal_positions_prob_diff.to("cpu"),
        "causal_positions_KL_loss": causal_positions_KL_loss.to("cpu"),
        "causal_positions_CE_loss_diff": causal_positions_CE_loss_diff.to("cpu"),
        "overall_total_effect_logits": overall_total_effect_logits.to("cpu"),
        "overall_total_effect_prob": overall_total_effect_prob.to("cpu"),
        "overall_total_effect_kl": overall_total_effect_kl.to("cpu"),
        "overall_total_effect_CE": overall_total_effect_CE.to("cpu"),
    }

In [15]:
clean_prompt, subject, target = sample_dataset(dataset)
original_fact, corrupted_facts, target = resample_ablation(clean_prompt, subject, target, n_noise_samples=10)

graft = get_causal_positions(original_fact, corrupted_facts, target)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 14.75 GiB total capacity; 13.28 GiB already allocated; 12.81 MiB free; 13.97 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [168]:
graft

{'causal_positions_logit_diff': tensor([[1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420, 1.2420],
         [1.2420, 1.2420, 1.242

## Bayesian Optimization with Probabilistic Reparametrization

In [16]:
!pip install scikit-optimize

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [19]:
from skopt import gp_minimize

In [132]:

def patch(
    corrupted_residual_component: Float[torch.Tensor, "batch pos d_model"],
    hook,
    pos,
    cache):
    corrupted_residual_component[:, pos, :] = cache[hook.name][:, pos, :]
    """
    Restore a patch in clean run. 
    """
    return corrupted_residual_component


def unembedding_function(residual_stack, cache, mlp=False) -> float:
    #we are only interested in applying the layer norm of the final layer on the final token
    #shape: [74, 5, 10, 1280] = n_layers, prompts, tokens, d_model
    z = cache.apply_ln_to_stack(residual_stack, layer = -1, mlp_input=mlp)
    z = z @ model.W_U
    return z


def similarity_loss(original_logits, patched_logits, alpha=1.0, beta=1.0):
    probs_original = F.softmax(original_logits, dim=-1)
    probs_grafted = F.softmax(patched_logits, dim=-1)
    kl_loss = F.kl_div(probs_grafted.log(), probs_original, reduction='batchmean')
    
    argmax_index = torch.argmax(original_logits)
    truth = torch.zeros_like(original_logits)
    truth[0, argmax_index] = 1
    
    ce = F.cross_entropy(patched_logits, truth)
    
    total_loss = alpha * kl_loss + beta * ce
    
    return total_loss
        
    
def continuous_objective(continuous_graft,
                         model,
                         clean_cache,
                         corrupted_tokens,
                         original_logits, 
                         all_activation_keys, 
                         l1_lambda = 0.01):
    
    continuous_graft = np.array(continuous_graft)
    continuous_graft = continuous_graft.reshape(len(all_activation_keys),
                                                corrupted_tokens.shape[-1])

    THRESH = 0.5  # Adjust the threshold based on your problem's characteristics

    binary_graft = np.where(continuous_graft > THRESH, 1, 0)
    # binary_graft = np.zeros_like(binary_graft)
    reference_loss = similarity_loss(original_logits, original_logits).item()

    fwd_hooks = []   
        
    for i in range(binary_graft.shape[0]):
        patch_name = all_activation_keys[i]
        for tok_pos in range(binary_graft.shape[1]):
            graft_val = binary_graft[i, tok_pos]
            if graft_val > 0:
                hook_fn = partial(patch, pos=tok_pos, cache=clean_cache)
                hook = (patch_name, hook_fn)
                fwd_hooks.append(hook)

    patched_logits = model.run_with_hooks(
        corrupted_tokens,
        fwd_hooks=fwd_hooks,
        return_type="logits"
    )
    patched_logits = patched_logits[:, -1, :]

    loss = similarity_loss(original_logits=original_logits, patched_logits=patched_logits)
    loss = loss.item()
    loss = loss - reference_loss
    # L1 regularization term to encourage sparsity
    l1_reg = l1_lambda * np.sum(np.abs(continuous_graft))
    total_loss = loss + l1_reg
    print(total_loss)
    return total_loss


    
def learn_graft(
             clean_prompt: str,
             corrupted_prompts: List[str],
             target: str):
    
    clean_tokens = model.to_tokens(clean_prompt, prepend_bos=True) 
    corrupted_tokens = model.to_tokens(corrupted_prompts, prepend_bos=True)
    corrupted_tokens = corrupted_tokens[0].unsqueeze(0)
    
    target_token_idx = model.to_tokens(target)[:,1] 
    clean_tokens, corrupted_tokens = pad_to_same_length(clean_tokens, corrupted_tokens)
    
    clean_tokens = clean_tokens.expand(corrupted_tokens.shape[0], -1)
    target_token_idx = target_token_idx.expand(corrupted_tokens.shape[0], -1)
    
    corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens, return_type="logits")
    clean_logits, clean_cache = model.run_with_cache(clean_tokens, return_type="logits")
    corrupted_logits = corrupted_logits[:,-1,:]
    
    
    #define all possible activations to ablate
    token_positions = torch.arange(corrupted_tokens.shape[-1], device=device)
    all_activation_keys = []
    for layer in range(model.cfg.n_layers): 
        for activation_patch in possible_patches:
            patch_name = f"blocks.{layer}.{activation_patch}"
            all_activation_keys.append(patch_name)

    # Define the bounds for the continuous relaxation (between 0 and 1 for each value)
    all_interventions = len(all_activation_keys) * clean_tokens.shape[-1]
    bounds = [(0, 1) for _ in range(all_interventions)]

    # Run Bayesian optimization
    result = gp_minimize(
        partial(continuous_objective, model=model, clean_cache=clean_cache,
                corrupted_tokens=corrupted_tokens, original_logits=corrupted_logits, 
                all_activation_keys=all_activation_keys),
        dimensions=bounds,
        n_calls=25  # Adjust the number of optimization calls as needed
    )

    return result
        

In [133]:
clean_prompt, subject, target = sample_dataset(dataset)
original_fact, corrupted_facts, target = resample_ablation(clean_prompt, subject, target, n_noise_samples=10)

graft = learn_graft(original_fact, corrupted_facts, target)

21.30443917274475
19.8521035861969
3.887194986343384
20.53722764015198
15.270883264541625
20.43558289527893
21.0517959690094
21.339812631607057
20.631496057510375
19.860386056900026
0.8797665023803711
21.006300287246702
20.064836587905884
21.168011331558226
19.868252267837523
4.523120489120483
21.106985177993774
6.567327432632446
21.653884439468385
21.36880478858948
21.413832559585572
2.254520254135132
20.59296280860901
20.88110520362854
21.072228021621704


In [134]:
np.array(graft["x"]).reshape(24,9)

array([[0, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 1, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 1, 0, 1, 0],
       [0, 1, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 1, 1, 1, 0],
       [1, 0, 0, 1, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1],
       [0, 1, 1, 0, 0, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 1, 0, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 0, 1, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 1],
       [1, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 1, 1, 0, 1, 1, 0, 0, 1],
       [1, 0, 0, 0, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 0]])